In [1]:
import requests

In [2]:
def main():
    url = "http://www.rcsb.org/pdb/rest/search"
    pdb_set = set()
    
    with open("combined/uniprots.txt", "r") as fh:
        for line in fh:
            uniprot = line.strip()
            query_text = """
<?xml version="1.0" encoding="UTF-8"?>

<orgPdbQuery>

<queryType>org.pdb.query.simple.UpAccessionIdQuery</queryType>

<description>Simple query for a list of Uniprot Accession IDs:</description>

<accessionIdList>""" + uniprot + """</accessionIdList>

</orgPdbQuery>"""
            header = {"Content-Type": "application/x-www-form-urlencoded"}

            response = requests.post(url, data=query_text, headers=header)

            if response.status_code == 200:
                result = response.text.split('\n')
                
                #print("For %s found %d PDB entries matching query." % (uniprot, len(result) - 1))
                #print(response.text)
                for element in result:
                    pdb_set.add(element.split(':', 1)[0])
            else:
                print("For %s failed to retrieve results" % uniprot)
            
        
    
    with open("combined/results.txt", "w") as fh:
        for pdb in pdb_set:
            fh.write(pdb + '\n')

In [3]:
if __name__ == '__main__':
    main()